In [1]:
import sys
sys.path.append('/renoir/carreres/CosmoSoft/snsim/')
import snsim

In [ ]:
import importlib
importlib.reload(snsim.survey_host)
importlib.reload(snsim.simu)
importlib.reload(snsim.generators)



In [2]:
sim = snsim.Simulator('/renoir/carreres/Documents/Paper_script/sim_input/config_paper_sim_test.yml',
                      print_config=True)

PARAMETERS USED IN SIMULATION

data:
    write_path: /renoir/carreres/mydatadec/MySim/paper_sim/output/
    write_format: ['parquet']
    sim_name: paper_ztf_test
survey_config:
    survey_file: /renoir/carreres/mydatadec/MySim/paper_sim/input_files/ztf_obsfile_maglimcat_test.parquet
    sig_psf: 0.0
    sig_zp: 0.0
    ra_size: 7.295
    dec_size: 7.465
    noise_key: ['maglimcat', 'mlim5']
    sub_field: rcid
    field_map: /renoir/carreres/Documents/Paper_script/sim_input/ZTF_CCD.dat
    add_data: ['maglimcat', 'infobits']
sim_par:
    z_range: [0.01, 0.17]
    nep_cut: [[5, -20, 50], [1, -20, 50, 'ztfr'], [1, -20, 50, 'ztfg'], [1, -20, 50, 'zti']]
    duration_for_rate: 10
snia_gen:
    rate: ztf20
    M0: jla
    sigM: 0.12
    model_config:
        model_name: salt2
        alpha: 0.14
        beta: 3.1
        dist_x1: N21
        dist_c: [-0.055, 0.023, 0.15]
cosmology:
    Om0: 0.32
    H0: 71.0
    Ode0: 0.6799999999999999
cmb:
    v_cmb: 0
mw_dust:
    model: ccm89
    rv: 3

In [3]:
sim.simulate()

      _______..__   __.         _______. __  .___  ___. 
     /       ||  \ |  |        /       ||  | |   \/   | 
    |   (----`|   \|  |       |   (----`|  | |  \  /  | 
     \   \    |  . `  |        \   \    |  | |  |\/|  | 
 .----)   |   |  |\   |    .----)   |   |  | |  |  |  | 
 |_______/    |__| \__|    |_______/    |__| |__|  |__| 
================================= Version : 0.4.5_dev ====== 
-----------------------------------------------------------

SIM NAME : paper_ztf_test
CONFIG FILE : /renoir/carreres/Documents/Paper_script/sim_input/config_paper_sim_test.yml
SIM WRITE DIRECTORY : /renoir/carreres/mydatadec/MySim/paper_sim/output/
SIMULATION RANDSEED : 22433

-----------------------------------------------------------

SURVEY FILE : /renoir/carreres/mydatadec/MySim/paper_sim/input_files/ztf_obsfile_maglimcat_test.parquet

First day in survey_file : 58288.17 MJD / 2018-06-19 04:07:30.000
Last day in survey_file : 58378.17 MJD / 2018-09-17 04:01:52.500

Survey effective du

RuntimeError: None of the object pass the cuts...

In [ ]:
sim.samples[0].plot_lc(34)

In [ ]:
from astropy.time import Time
t = Time(59500, format='mjd')

In [ ]:
import astropy.units as aunit

In [ ]:
t + 1 * aunit.day

In [ ]:
sim.survey.start_end_days